Nos conectamos a la DB de Langfuse y hacemos queries sobre la tabla de trazas

Podemos descargarnos los datos totales de trazas pero solo con las columnas que nos importan. 

Para poder estimar cuales son los tags mas caros y demas informacion relevante necesitamos el script que habia hecho hace unos meses de estimación de costes. En este caso no lo voy a hacer para un tenant sino para un tag o serie de tags.

Preguntas:

* Cuantas trazas hay
* Cuantas trazas de los ultimos X dias que tienen la palabra prod o dev
* Cuanto nos gastamos en prod
* Cuanto nos gastamos en dev
* Cuanto nos gastamos en el resto de trazas
* Rankear las trazas no prod o dev

Cuan importante es este tema?
Queremos automatizarlo? En tal caso, como queremos hacerlo?
Queremos que le pase la tarea a Nekane?

----


Ventajas de hacer con script:

* Si tenemos claro lo que queremos ver, al tenerlo de forma programatica pues se ejecuta automaticamente y podemos "construir" sobre ello

Desventajas:
* Lleva tiempo de programar
* Va a cambiar lo que queremos ver?
* Vamos a dejar de usar langfuse?


-----

TODO: Hacer un notebook con algunas de estas preguntas resueltas, pedir un meeting con Adma y Juan mañana




In [8]:
import pandas as pd
from datetime import date

from aily_ai_brain.common.secrets_manager import get_aws_secret_key
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session, sessionmaker
from aily_py_commons.aily_logging import aily_logging as logging
from aily_py_commons.io.env_vars import (
    INFRASTRUCTURE_PROD,
    AilySettings,
)

## Connect to DB

In [4]:
def create_db_session_standard(tenant_name: str, environment: str):
    """
    Retrieves the database info for the given tenant using AWS credentials,
    then, creates a new database session for it.
    """
    secret_name = f"aily/{tenant_name}/{environment}/rds-langfuse/dbo"
    creds = {k: get_aws_secret_key(secret_name, k) for k in ["username", "password", "host", "port", "dbName"]}

    dialect_driver = "postgresql+psycopg2://"
    engine_url = (
        f"{dialect_driver}{creds['username']}:{creds['password']}@{creds['host']}:{creds['port']}/{creds['dbName']}"
    )
    sql_engine = create_engine(engine_url)
    sql_session = sessionmaker(bind=sql_engine)
    return sql_session

In [5]:
AilySettings(INFRASTRUCTURE_PROD)

2024-07-15 18:57:07 CEST+0200 - WARNING - aily-logging: You are using a dictionary to configure AilySettings. This is not recommended.Please use .env file to set your env variables in your environment.


AilySettings(AILY_PREFIX='aily', AILY_ENV='prod', AILY_TENANT='infrastructure', AWS_REGION='eu-central-1', AWS_PROFILE='aws-infrastructure')

In [6]:
tenant_name = "infrastructure"
environment = "prod"
tenant_session = create_db_session_standard(tenant_name, environment)

/var/folders/47/gybx85kd6f147h3rp8wgdz9w0000gn/T/ipykernel_95149/3766700193.py:7: DeprecationWarning: Please use `aily_data_aws.aws.secrets.get_secret()`
  creds = {k: get_aws_secret_key(secret_name, k) for k in ["username", "password", "host", "port", "dbName"]}


## Download data

In [9]:
def get_trace_sessions(
    session: Session, columns: list[str] = [], ids: list[int] = [], from_date: date = None
) -> pd.DataFrame:
    """
    Return all trace sessions
    """
    if not columns:
        selected_columns = "*"
    else:
        selected_columns = ", ".join(f'"{col}"' for col in columns)

    conditions = []
    params = {}

    if ids:
        conditions.append("id = ANY (ARRAY[:ids])")
        params["ids"] = ids

    if from_date:
        conditions.append("created_at >= :from_date")
        params["from_date"] = from_date

    if conditions:
        query_where = " WHERE " + " AND ".join(conditions)
    else:
        query_where = ""

    query = text(
        f"""
            SELECT {selected_columns}
            FROM public.trace_sessions
            {query_where}
            """
    )

    result = session.execute(query, params=params)

    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df

In [11]:
with tenant_session() as session:
    df_trace_session_ids = get_trace_sessions(session, columns=["id"])

In [12]:
df_trace_session_ids

,id
0,KYdlzJ3IWU8gagGUy6g3kg
1,CMW-GNvmRQVIDrDpro-oDQ
2,W7NqR4ZMwh1iIK4tZwMMfA
3,_rGcF_UVvVnNvL-J1aOoHQ
4,uEn4wNaP7BYs8MXuJHklUw
...,...
46024,coreproduct-pQh_DXXvGGysR5lW-n1XmA
46025,coreproduct-M1rAegkzFxnVPbk2H0t_vw
46026,coreproduct-6IoMpUMAGhcF_MJVsylhug
46027,coreproduct-mqH07991PmdrAbYGrgYT5g
